In [1]:
from collections import Counter
from model import RNNModule
from music21 import note, chord, instrument, stream
import numpy as np
import pickle
import torch

In [2]:
# параметры модели
batch_size     = 20
sequence_size  = 100
embedding_size = 200
hidden_size    = 400

## Генерация примеров

In [3]:
# вновь загружаем ноты из файлов и делаем словари

notes = []

with open('data/notes_ibi', 'rb') as f:
    notes += pickle.load(f)
    
with open('data/notes_classic', 'rb')as f:
    notes += pickle.load(f)
    
dict_notes = Counter(notes)

sorted_notes = sorted(dict_notes, key=dict_notes.get, reverse=True)

int_to_note = {i : n for i,n in enumerate(sorted_notes)}
note_to_int = {n : i for i,n in enumerate(sorted_notes)}

n_notes = len(int_to_note)

In [4]:
# создаём модель
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = RNNModule(sequence_size, hidden_size, embedding_size, batch_size, n_notes)
net = net.to(device)

# загружаем состояние модели 282 эпохи(45000 итерации)
net.load_state_dict(torch.load("data/model_states/model-45000.pth"))

np.random.seed(42)

In [5]:
def predict(device, net, first_notes, top_prob, file_output):
    """ функция генерирует midi файл, используя обученную модель """
    
    net.eval()
    
    midi_notes = first_notes
    
    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    
    # запускаем модель на первых нотах
    for n in midi_notes:
        ix = torch.tensor([[note_to_int[n]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    # выбираем k элементов с набольшей вероятностью
    _, top_ix = torch.topk(output[0], k=top_prob)
    
    # выбираем 1 элемент из k, он будет следующим в последовательности
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])
  
    midi_notes.append(int_to_note[choice])

    # запускаем модель и генерируем композицию длины 500 + кол-во начальных нот
    
    for _ in range(250):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_prob)
        
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        
        midi_notes.append(int_to_note[choice])
        
    # добавляем смещение, чтобы ноты не накладывались
    offset = 0
    
    
    output_notes = []

    # генерируем последовательность нот, которая будет основой midi файла
    for pattern in midi_notes:        
        # добавляем аккорд (ноты разделены точкой)
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # добавляем ноту
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.65
        
    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=file_output)

In [6]:
# ноты и аккорды, с которых будет генерироваться мелодия
print(int_to_note[444], int_to_note[0])
print(int_to_note[321])

5.6.9 C5
2.6.8


In [7]:
# генерируем мелодии
predict(device,net, [int_to_note[0], int_to_note[1]], 1, "data/samples/sample1.mid")
predict(device,net, [int_to_note[555]], 4, "data/samples/sample2.mid")